In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import tensorflow as tf
import numpy as np
import time
import json

from model import Model_S2VT
from data_generator import Data_Generator
from inference_util import Inference

import inception_base
import configuration

FLAGS = tf.app.flags.FLAGS

tf.flags.DEFINE_integer("batch_size", 64,
                       "Batch size of train data input.")
tf.flags.DEFINE_string("checkpoint_model", None,
                       "Model Checkpoint to use.")
tf.flags.DEFINE_integer("max_captions", None,
                       "Maximum number of captions to generate")
tf.flags.DEFINE_integer("max_len_captions", None,
                       "Maximum length of captions to generate")
tf.flags.DEFINE_string("dataset", "test",
                       "Dataset to use")

In [2]:
FLAGS.checkpoint_model = "./s2vt_models/train/model-10164"

In [3]:
data_config = configuration.DataConfig().config
data_gen = Data_Generator(processed_video_dir = data_config["processed_video_dir"],
                         caption_file = data_config["caption_file"],
                         unique_freq_cutoff = data_config["unique_frequency_cutoff"],
                         max_caption_len = data_config["max_caption_length"])

data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])
#data_gen.build_dataset()

In [ ]:
data_gen.sa

In [12]:
if FLAGS.max_len_captions:
    max_len = FLAGS.max_len_captions
else:
    max_len = data_config['max_caption_length']

model_config = configuration.ModelConfig(data_gen).config
model = Model_S2VT( num_frames = model_config["num_frames"],
                    image_width = model_config["image_width"],
                    image_height = model_config["image_height"],
                    image_channels = model_config["image_channels"],
                    num_caption_unroll = model_config["num_caption_unroll"],
                    num_last_layer_units = model_config["num_last_layer_units"],
                    image_embedding_size = model_config["image_embedding_size"],
                    word_embedding_size = model_config["word_embedding_size"],
                    hidden_size_lstm1 = model_config["hidden_size_lstm1"],
                    hidden_size_lstm2 = model_config["hidden_size_lstm2"],
                    vocab_size = model_config["vocab_size"],
                    initializer_scale = model_config["initializer_scale"],
                    learning_rate = model_config["learning_rate"])
model.build()

gen_caption = []

infer_util = Inference(model,data_gen.word_to_idx,data_gen.idx_to_word)

In [13]:
sess = tf.Session()

In [14]:
saver = tf.train.Saver(max_to_keep=20,keep_checkpoint_every_n_hours=0.5)
if FLAGS.checkpoint_model:
    model_path = FLAGS.checkpoint_model
else:
    model_path = tf.train.latest_checkpoint(data_config["checkpoint_dir"])

if model_path != None:
    print("Restoring weights from %s" %model_path)
    saver.restore(sess,model_path)
else:
    print("No checkpoint found. Exiting")

Restoring weights from ./s2vt_models/train/model-10164
INFO:tensorflow:Restoring parameters from ./s2vt_models/train/model-10164


In [15]:
dataset_dir = "../dataset/MSVD_processed/test/"

In [16]:
data_gen.dataset[FLAGS.dataset] = [{"file_name":i,"path":os.path.join(dataset_dir,i)} for i in os.listdir(dataset_dir) if "avi.npy" in i]

In [17]:
if FLAGS.max_captions:
    max_iter = FLAGS.max_captions
else:
    max_iter = len(data_gen.dataset[FLAGS.dataset])+10 #+10 is just to be safe ;)

iter = 0
btch = 0

In [18]:
video_paths = {i["file_name"]:i["path"] for i in data_gen.dataset[FLAGS.dataset]}
video_files = list(video_paths.keys())

In [19]:
for btch in range(0,len(video_files),FLAGS.batch_size):
    print("Processing batch %d" %(int(btch/FLAGS.batch_size)+1))
    start = btch
    end = min(len(video_files),btch+FLAGS.batch_size)
    dataset={}
    dataset["video"] = np.asarray([np.load(video_paths[video_files[i]]) for i in range(start,end)])
    dataset["path"] = [video_paths[video_files[i]] for i in range(start,end)]
    dataset["file"] = [video_files[i] for i in range(start,end)]
    dataset["gen_caption"] = infer_util.generate_caption_batch(sess,dataset["video"],max_len=max_len)
    for i in range(len(dataset['gen_caption'])):
        dictionary = {}
        dictionary["gen_caption"] = dataset['gen_caption'][i]
        dictionary["file_name"] = dataset['file'][i]
        dictionary["path"] = dataset['path'][i]
        gen_caption.append(dictionary)
        iter+=1
        if iter >= max_iter:
            break
    if iter >= max_iter:
        break
with open(os.path.join(data_config["result_dir"],"generated_caption_msvd.json"),"w") as fl:
    fl.write(json.dumps(gen_caption, indent=4, sort_keys=True))

Processing batch 1
Processing batch 2
Processing batch 3


In [20]:
basename = "./rgb_test_features/"

In [21]:
fl = open("../ignore/S2VT/S2VT_results.txt","w")

In [22]:
for i in range(len(gen_caption)):
    fl.write(os.path.join(basename,gen_caption[i]['file_name'])+"\n")
    fl.write(gen_caption[i]['gen_caption']+"\n\n")

In [23]:
fl.close()

In [ ]:
gen_caption[8]

In [ ]:
x = np.load("../dataset/MSVD_processed/test/VyBXcQ1v_cw_7_15.avi.npy")

In [ ]:
y = np.load("../dataset/MSVD_processed/test/VyBXcQ1v_cw_7_15.avi.mask.npy")

In [2]:
import numpy as np

In [3]:
x = np.load("/home/ozym4nd145/Coding/Notebook/SURA/ignore/S2VT/rgb_test_features/X98fLZHIzWQ_15_20.avi.npy")

In [ ]:
y.dtype

In [ ]:
y

In [ ]:
sess.close()

In [6]:
val_files = ["X98fLZHIzWQ_15_20.avi.npy","yREFkmrrYiw_51_57.avi.npy","xaPepCVepCg_35_46.avi.npy","YRSOc193Pxc_68_73.avi.npy","xBePrplM4OA_6_18.avi.npy","YS1mzzhmWWA_9_16.avi.npy","xb-Nacm-pKc_78_82.avi.npy","ysTmUTQ5wZE_17_45.avi.npy","xBW_uR3kGr4_42_55.avi.npy","yU5sxW9bErQ_0_17.avi.npy","xCFCXzDUGjY_5_9.avi.npy","_yUiusXxfWI_150_165.avi.npy","XdB0pVjFYoU_1_8.avi.npy","YUlG_PKzpvc_0_8.avi.npy","xdhLQCYQ-nQ_50_63.avi.npy","yvfhk1kwKls_103_113.avi.npy","xEDCfcMZlZY_37_41.avi.npy","ywHBKayhyvQ_19_28.avi.npy","xEDCfcMZlZY_91_100.avi.npy","ywMLb2VfHbI_13_20.avi.npy","XEOH3t5AYY0_22_29.avi.npy","YwmUgVrUJ4I_0_15.avi.npy","_xf24TYgbuY_14_20.avi.npy","YXixZrSxefk_17_48.avi.npy","xfRIRSWy0y0_10_20.avi.npy","YXixZrSxefk_49_59.avi.npy","xGdoi3W3Z4o_2_40.avi.npy","YXixZrSxefk_6_15.avi.npy","xgIIcPSh4EU_0_6.avi.npy","yYA7YXIKbg4_9_14.avi.npy","Xh6rdpJNaf0_0_15.avi.npy","yyxtyCaEVqk_250_264.avi.npy","xkNDE1JL6Z8_50_59.avi.npy","yyxtyCaEVqk_321_328.avi.npy","xlPyHMkpaQY_1_7.avi.npy","yyxtyCaEVqk_329_360.avi.npy","X_NLV2KCnIE_60_70.avi.npy","YZ0-6hdQ0mU_0_3.avi.npy","XNV7hKVu-Xg_40_47.avi.npy","z0Si1XxMibg_0_30.avi.npy","XNzXXiE6qR4_0_10.avi.npy","z0zb--BOhDY_16_22.avi.npy","XOAgUVVwKEA_8_20.avi.npy","Z10_CDPQKKs_1_6.avi.npy","xOQn1z_oWDY_30_39.avi.npy","Z19zFlPah-o_6_11.avi.npy","Xp6rxW4hFFo_32_40.avi.npy","z1PEyzk4ADU_8_18.avi.npy","xpOYXbMDDBY_1_20.avi.npy","z2kUc8wp9l8_40_46.avi.npy","xPR0xFgCAZY_17_27.avi.npy","Z3m9V8qPXhQ_13_24.avi.npy","xSmfK498DjI_1_20.avi.npy","Z3rj9LbCOaM_0_12.avi.npy","xtbsD3PUua4_174_185.avi.npy","Z3XarHZ3HpY_10_17.avi.npy","xtbsD3PUua4_84_94.avi.npy","z8dcUzdi2bw_3_16.avi.npy","xTd_IdIcB4s_2_10.avi.npy","Z8rTzW9Gb6Y_116_123.avi.npy","XtQdAPV2UZs_4_11.avi.npy","z9qbQX4J_2g_0_14.avi.npy","Xvy21KrDv8I_0_10.avi.npy","za-9mBZyNfQ_330_336.avi.npy","Xwrl4htUPjo_22_38.avi.npy","Z-An1QQ66f0_13_25.avi.npy","Xxcv5jvRmcg_39_46.avi.npy","zbAk0gX7kas_16_24.avi.npy","xxHx6s_DbUo_121_128.avi.npy","zBrC1GmA0Qg_26_33.avi.npy","xxHx6s_DbUo_158_161.avi.npy","ZbtpcGi2DWY_161_170.avi.npy","xxHx6s_DbUo_162_165.avi.npy","ZbtpcGi2DWY_56_62.avi.npy","xxHx6s_DbUo_173_177.avi.npy","ZbzDGXEwtGc_6_15.avi.npy","xxHx6s_DbUo_182_186.avi.npy","zCf8NWJ8kzA_47_52.avi.npy","xxHx6s_DbUo_202_207.avi.npy","ZdlG8fjGJlo_78_87.avi.npy","xxHx6s_DbUo_216_222.avi.npy","ZdNP0dXCx2I_78_90.avi.npy","xxHx6s_DbUo_240_248.avi.npy","ZDoy9S82ETA_5_15.avi.npy","xxHx6s_DbUo_32_36.avi.npy","zFIn8DeV5PM_20_33.avi.npy","xxHx6s_DbUo_36_40.avi.npy","zfQOH4UGU_I_1_10.avi.npy","xxHx6s_DbUo_41_44.avi.npy","Zg0p5TXi_Cc_30_46.avi.npy","xxHx6s_DbUo_49_56.avi.npy","zH4v_DClIoc_3_28.avi.npy","xxHx6s_DbUo_57_61.avi.npy","zhLcONtNkew_16_21.avi.npy","xxHx6s_DbUo_82_86.avi.npy","zHy7pM0U49w_103_109.avi.npy","xxHx6s_DbUo_98_101.avi.npy","zHy7pM0U49w_110_116.avi.npy","xXlWsBhFvis_6_10.avi.npy","ZK4W-2ifl6I_1_28.avi.npy","xy9LLUUZ6ic_50_60.avi.npy","zkTn5Ef1Oig_70_75.avi.npy","XZ0Ch_ClBfo_120_127.avi.npy","zkTn5Ef1Oig_71_75.avi.npy","XzCcRzEa83U_1_8.avi.npy","ZL7P_XmSnEY_205_209.avi.npy","XzCcRzEa83U_39_46.avi.npy","zlS1_zBYluY_15_21.avi.npy","Y4BeJ-64294_20_30.avi.npy","ZlX_Gy4HP2E_38_55.avi.npy","Y4gaJn0wlDE_0_10.avi.npy","zMg9dChF97A_12_38.avi.npy","y8SDRc4IOjs_94_99.avi.npy","ZN2_czSBSD0_240_250.avi.npy","yAD_TS5L2d4_4_11.avi.npy","ZnLgsasu82g_71_81.avi.npy","Ya-MkA187c8_36_45.avi.npy","-zOrV-5vh1A_69_76.avi.npy","YAud4eS3DoA_54_65.avi.npy","Zoz0HcCB1vU_0_10.avi.npy","yBK02O1Oewc_20_35.avi.npy","zpgW7m7_LZw_2_15.avi.npy","ybVb3t560oY_0_9.avi.npy","z_qf7tOnHVg_81_89.avi.npy","yC4eEuURH8c_19_28.avi.npy","Zqmx-nXhLAg_0_10.avi.npy","yd8jh9QYfEs_4_10.avi.npy","zr9OeyfLPPY_4_9.avi.npy","YEmS8ZQIhTA_7_20.avi.npy","zS50h-a8RTg_3_9.avi.npy","YersIyzsOpc_15_30.avi.npy","zSPBC8EO6dY_122_126.avi.npy","yfMTsYcLt10_0_7.avi.npy","zSPBC8EO6dY_132_140.avi.npy","yFPHhRat6bc_160_210.avi.npy","zSPBC8EO6dY_64_73.avi.npy","yg4aNiO1JnI_111_114.avi.npy","zSPBC8EO6dY_97_110.avi.npy","Yh2N6ZtRBdU_110_120.avi.npy","ZsrPKtx0qyg_7_15.avi.npy","-YI0cxuNcq8_262_272.avi.npy","zTn-nNj5Bng_61_77.avi.npy","yId1rg5-ac0_40_50.avi.npy","zTn-nNj5Bng_8_19.avi.npy","YJ2aGe7CLBo_25_35.avi.npy","zulPFoY64wE_26_33.avi.npy","Ylb4pnW-jP4_102_113.avi.npy","zuYZ5kPatJE_78_116.avi.npy","ymC2bNi6-Is_9_19.avi.npy","zv2RIbUsnSw_159_162.avi.npy","YmXCfQm0_CA_109_120.avi.npy","zv2RIbUsnSw_335_341.avi.npy","YmXCfQm0_CA_140_156.avi.npy","ZvJvNcukZ4w_0_10.avi.npy","YmXCfQm0_CA_277_284.avi.npy","zWVQImPY2Cc_27_44.avi.npy","YmXCfQm0_CA_50_57.avi.npy","_ZwwKOzpt2I_69_76.avi.npy","YmXCfQm0_CA_68_75.avi.npy","zxB4dFJhHR8_1_9.avi.npy","YmXCfQm0_CA_7_16.avi.npy","ZxqkAGNyo8A_12_22.avi.npy","YndZ9YulpTo_61_76.avi.npy","ZYaxg5NZW_g_21_27.avi.npy","yNy9jTeolUk_8_12.avi.npy","zYcY4mjLpxU_104_118.avi.npy","yOnRm3TP6hM_0_12.avi.npy","zYcY4mjLpxU_45_55.avi.npy","yPNFVj-pISU_105_115.avi.npy","zzit5b_-ukg_5_20.avi.npy"]

In [7]:
val_files = set(val_files)

{'-YI0cxuNcq8_262_272.avi.npy',
 '-zOrV-5vh1A_69_76.avi.npy',
 'X98fLZHIzWQ_15_20.avi.npy',
 'XEOH3t5AYY0_22_29.avi.npy',
 'XNV7hKVu-Xg_40_47.avi.npy',
 'XNzXXiE6qR4_0_10.avi.npy',
 'XOAgUVVwKEA_8_20.avi.npy',
 'XZ0Ch_ClBfo_120_127.avi.npy',
 'X_NLV2KCnIE_60_70.avi.npy',
 'XdB0pVjFYoU_1_8.avi.npy',
 'Xh6rdpJNaf0_0_15.avi.npy',
 'Xp6rxW4hFFo_32_40.avi.npy',
 'XtQdAPV2UZs_4_11.avi.npy',
 'Xvy21KrDv8I_0_10.avi.npy',
 'Xwrl4htUPjo_22_38.avi.npy',
 'Xxcv5jvRmcg_39_46.avi.npy',
 'XzCcRzEa83U_1_8.avi.npy',
 'XzCcRzEa83U_39_46.avi.npy',
 'Y4BeJ-64294_20_30.avi.npy',
 'Y4gaJn0wlDE_0_10.avi.npy',
 'YAud4eS3DoA_54_65.avi.npy',
 'YEmS8ZQIhTA_7_20.avi.npy',
 'YJ2aGe7CLBo_25_35.avi.npy',
 'YRSOc193Pxc_68_73.avi.npy',
 'YS1mzzhmWWA_9_16.avi.npy',
 'YUlG_PKzpvc_0_8.avi.npy',
 'YXixZrSxefk_17_48.avi.npy',
 'YXixZrSxefk_49_59.avi.npy',
 'YXixZrSxefk_6_15.avi.npy',
 'YZ0-6hdQ0mU_0_3.avi.npy',
 'Ya-MkA187c8_36_45.avi.npy',
 'YersIyzsOpc_15_30.avi.npy',
 'Yh2N6ZtRBdU_110_120.avi.npy',
 'Ylb4pnW-jP4_102_113